In [2]:
import logging
import time
from datetime import datetime

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
from distributed import LocalCluster, Client

In [3]:
cluster = LocalCluster(n_workers=2, threads_per_worker=4)#silence_logs=logging.ERROR)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33215 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 8 Memory: 67.47 GB


In [4]:
# allow many retries
dask.config.config['distributed']['scheduler']['allowed-failures'] = 10

In [5]:
df = dd.read_parquet('../data/resaved-gps-points', engine='pyarrow')
# df = df.partitions[1217:]#.loc[:1_000_000, :].repartition(npartitions=8).persist()
# df.head()

In [8]:
tmp_dd = df

In [10]:
@dask.delayed
def reverse_pdf(pdf):
    '''delayed function to reverse a pandas dataframe'''
    return pdf[::-1]

ds = tmp_dd.to_delayed() # one delayed object per partition
ds = [reverse_pdf(d) for d in ds] # reverse each partition
ds = reversed(ds) # reverse the order of the partitions
tmp_dd_reversed = dd.from_delayed(ds) # construct a new dask dataframe

In [11]:
df = tmp_dd_reversed

In [12]:
df['geohash'] = df.apply(lambda row: geohash.encode(row.latitude, row.longitude), axis=1, meta='object')

In [10]:
deal.to_parquet('../data/geohashed_gps_points3/1217_v2')

AttributeError: 'Series' object has no attribute 'to_parquet'

In [8]:
df.describe().compute()

,latitude,longitude
count,1.540396e+06,1.540396e+06
mean,7.043155e+01,2.311681e+01
std,1.905008e+00,1.482495e+01
min,6.978186e+01,-1.771436e+02
25%,6.981549e+01,2.095028e+01
50%,7.002694e+01,2.097784e+01
75%,7.009438e+01,2.348621e+01
max,9.000000e+01,1.794087e+02


In [ ]:
%%time 
t0 = time.time()
dfs = []

with open(f'{datetime.now()}_log.txt', 'w') as f:
    for i in range(900, 1218):
        try:
            temp_df = df.partitions[i]
            temp_df['geohash'] = temp_df.apply(lambda row: geohash.encode(row.latitude, row.longitude), axis=1, meta='object')
            temp_df.to_parquet(f'../data/geohashed_gps_points3/{i:04d}')
        except Exception as e:
            msg = str(e) + f'\n partition {i:04d} \n ---'
            print(msg)
            f.write(msg)
dt_hr = (time.time() - t0)/60/60